# 4.4 增加快捷链接

接下来，让我们讨论快捷连接背后的概念，它也可称为跳过或剩余连接。
最初，快捷连接是为计算机视觉中的深度网络（特别是残差网络）而提出的，以缓解梯度消失带来的挑战。
梯度消失问题是指梯度（在训练过程中引导权重更新）随着层反向传播而逐渐变小的问题，使得训练早期层变得困难，
如图 4.12 所示。

**图 4.12 是由 5 层组成的不带快捷连接（左侧）和带快捷连接（右侧）的深度神经网络之间的比较。
快捷连接涉及将层的输入添加到其输出，从而有效地创建绕过某些层的备用路径。
图 1.1 中所示的梯度表示每层的平均绝对梯度，我们将在下面的代码示例中计算。**

![fig4.12](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-4-12.jpg?raw=true)

如图 4.12 所示，快捷连接通过跳过一个或多个层，为梯度在网络中流动创造了一条更短的备用路径。这是通过将一个层的输出加到后面层的输出上来实现的。
这就是为什么这些连接也称为跳过连接。
它们在训练过程中的反向传播中起着至关重要的作用，以保持梯度的流动。

在下面的代码示例中，我们实现了图 4.12 所示的神经网络，看看如何在前向方法中添加快捷连接：

### 代码示例 4.5 说明快捷方式连接的神经网络

In [1]:
import torch
import torch.nn as nn
from torch.nn import GELU


class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            # 实现5层神经网络
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
       ])
    def forward(self, x):
        for layer in self.layers:
            # 计算当前层的输出
            layer_output = layer(x)
            # 检查是否可以应用快捷连接
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x

这段代码实现了一个 5 层的深度神经网络，每层由一个线性层和一个 GELU 激活函数组成。
在前向传递中，我们迭代地通过各层传递输入，如果 self.use_shortcut 属性设置为 True，则可以选择添加图 4.12 中所示的快捷方式连接。

让我们使用此代码来初始化一个没有快捷连接的神经网络。
在这里，每个层都将被初始化，以便它接受具有 3 个输入值的示例，并返回 3 个输出值。
最后一层返回单个输出值：

In [2]:
layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
torch.manual_seed(123) # 为初始权重指定随机种子以确保可重现性
model_without_shortcut = ExampleDeepNeuralNetwork(
     layer_sizes, use_shortcut=False
)

接下来，我们将通过下面代码实现一个函数，在模型的反向传递中计算梯度：

In [3]:
def print_gradients(model, x):
    # Forward pass
    output = model(x)
    target = torch.tensor([[0.]])
   
    # 根据目标和输出的接近程度计算损失
    # 输出是怎样的形式
    loss = nn.MSELoss()
    loss = loss(output, target)

    # 反向传播以计算梯度
    loss.backward()

    for name, param in model.named_parameters():
        if 'weight' in name:
           # 输出权重的平均绝对梯度
           print(f"{name} has gradient mean of {param.grad.abs().mean()}")

在上面的代码中，我们指定了一个损失函数，用于计算模型输出与用户指定的目标（此处为简单起见，值为 0）的接近程度。
然后，当调用 loss.backward() 时，PyTorch 计算模型中每一层的损失梯度。
我们可以通过 model.named_parameters() 迭代权重参数。
假设给定层有一个 3×3 的权重参数矩阵。
在这种情况下，该层将具有 3×3 梯度值，我们打印这些 3×3 梯度值的平均绝对梯度，以获得每层的单个梯度值，以便更容易地比较各层之间的梯度。

简而言之，.backward() 方法是 PyTorch 中计算模型训练期间所需的损失梯度的一种便捷方法，无需我们自己实现梯度计算的数学运算，从而使深度神经网络的使用变得更加容易。
如果不熟悉梯度和神经网络训练的概念，推荐阅读附录 A 中的 A.4“自动微分变得容易”和 A.7 “典型训练循环”部分。

现在让我们使用 print_gradients 函数，并将其应用于没有跳跃连接的模型：

In [4]:
print_gradients(model_without_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.0002017411752603948
layers.1.0.weight has gradient mean of 0.00012011770741082728
layers.2.0.weight has gradient mean of 0.0007152437465265393
layers.3.0.weight has gradient mean of 0.0013988513965159655
layers.4.0.weight has gradient mean of 0.005049604922533035


输出如下所示：

layers.0.0.weight has gradient mean of 0.00020173587836325169 \
layers.1.0.weight has gradient mean of 0.0001201116101583466 \
layers.2.0.weight has gradient mean of 0.0007152041653171182 \
layers.3.0.weight has gradient mean of 0.001398873864673078 \
layers.4.0.weight has gradient mean of 0.005049646366387606

从 print_gradients 函数的输出可以看出，梯度从最后一层（layers.4）到第一层（layers.0）逐渐变小，这种现象被称为梯度消失问题。

现在，让我们实例化一个带有跳跃连接的模型，看看它与之前的模型有何不同：

In [5]:
torch.manual_seed(123)
model_with_shortcut = ExampleDeepNeuralNetwork(
layer_sizes, use_shortcut=True
)
print_gradients(model_with_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.22186796367168427
layers.1.0.weight has gradient mean of 0.207092747092247
layers.2.0.weight has gradient mean of 0.32923877239227295
layers.3.0.weight has gradient mean of 0.2667771875858307
layers.4.0.weight has gradient mean of 1.3268063068389893


输出如下所示：

layers.0.0.weight has gradient mean of 0.22169792652130127 \
layers.1.0.weight has gradient mean of 0.20694105327129364 \
layers.2.0.weight has gradient mean of 0.32896995544433594 \
layers.3.0.weight has gradient mean of 0.2665732502937317 \
layers.4.0.weight has gradient mean of 1.3258541822433472 \

正如我们所看到的，根据输出，最后一层（layers.4）仍然比其他层具有更大的梯度。
然而，随着我们向第一层（layers.0）进展，梯度值变得更加稳定，并没有缩减到极小的数值。

总之，快捷连接对于克服深度神经网络中梯度消失问题所带来的限制非常重要。
快捷连接是大型模型（例如 LLMs）的核心构建块，当我们在下一章中训练 GPT 模型时，它们将通过确保跨层梯度流的一致来帮助促进更有效的训练。

介绍完快捷连接后，我们现在将在下一节的transfomer模块中连接所有先前介绍的概念（层归一化、GELU 激活函数、前馈模块和快捷连接），这是我们编码所需的最终构建块GPT 架构。